<a href="https://colab.research.google.com/github/hawc2/contentdm-iiif-api/blob/main/ContentDM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This Python script queries the ContentDM API and downloads metadata Temple University Libraries' Digital Collections. 

# Install Packages


In [24]:
from google.colab import files
%load_ext google.colab.data_table
import requests, csv
import json
import pandas as pd
from pandas.io.json import json_normalize
import cv2
import os
import urllib.request
import urllib3
from urllib3 import request

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


# Query ContentDM

## Download Temple Metadata for Digital collections

In [25]:
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/manifest.json'

In [26]:
resp = urllib.request.urlopen(manifestUrl)
data = resp.read().decode("utf-8")
data = json.loads(data)

In [27]:
df = pd.json_normalize(data['collections'])
del df['@type']
df.rename(columns = {'@id':'ID', 'label':'Label'}, inplace = True)
df.head()

,ID,Label
0,https://cdm16002.contentdm.oclc.org/iiif/info/...,Allied Posters of World War I
1,https://cdm16002.contentdm.oclc.org/iiif/info/...,Ambler Campus History in Photographs
2,https://cdm16002.contentdm.oclc.org/iiif/info/...,City Parks Association Photographs
3,https://cdm16002.contentdm.oclc.org/iiif/info/...,Frank G. Zahn Railroad Photograph Collection
4,https://cdm16002.contentdm.oclc.org/iiif/info/...,Franklin H. Littell Papers


In [28]:
df.to_csv (r'manifest.csv', index = False, header=True)

In [29]:
files.download('manifest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Query a Collection

The manifestURL variable determines what collection, such as WWII Posters, are queried. 

In [30]:
collectionUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/p1.json'

In [31]:
responseManifest = requests.get(collectionUrl)
m = json.loads(responseManifest.text)

In [32]:
csv_out = csv.writer(open('collection.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])

44

In [33]:
for i in m['manifests']:
    title = date = contributor = description = subject = ''
    
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)

    title = manifestItem['metadata'][0]['value']
    date = manifestItem['metadata'][1]['value']
    contributor = manifestItem['metadata'][2]['value']
    description = manifestItem['metadata'][3]['value']
    subject = manifestItem['metadata'][4]['value']
    csv_out.writerow([title, date, contributor, description, subject])    

In [34]:
files.download('collection.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
df = pd.read_csv('collection.csv')
df.head()

,Title,Date,Contributor,Description,Subject
0,Sottoscrivete al prestito,[ca. 1917],"Capranesi, Giovanni, 1852-1921, artist.; Istit...",Allegorical depiction of Italy as a female wea...,"Date of publication from Rawls, Walton, Wake u..."
1,Step into your place,Poster (Great Britain. Parliamentary Recruitin...,1915,Great Britain. Parliamentary Recruiting Commit...,British recruiting poster. Illustration depic...
2,Indian Ink,1916,"Pereira, Harold B., artist.;",Poster advertising the Christmas 1916 edition ...,"Signed: Harold B. Pereira, 16 [i.e. 1916?]. Or..."
3,Per la libertà e la civiltà del mondo,1918,"Dudovich, Marcello, 1878-1962, artist.; Atelie...",Italian poster urging subscription to the nati...,Signed: M. Dudovich. Date of publication from ...
4,The watchers of the seas,[ca. 1914-1918],"Raven-Hill, L. (Leonard), 1867-1942, artist.; ...",British naval recruiting poster that asks for ...,"Signed: ""L. Ravenhill."" Date based on the date..."
